In [1]:
Q_TKN = "<Q>"
A_TKN = "<A>"
BOS = '</s>'
EOS = '</s>'
UNK = '<unk>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

In [2]:
import os

# 작업 디렉터리 변경
new_work_dir = './app'
os.chdir(new_work_dir)
print(f"작업 디렉터리가 '{new_work_dir}'로 변경되었습니다.")

작업 디렉터리가 './app'로 변경되었습니다.


In [3]:
import pandas as pd
from datasets import Dataset
from transformers import PreTrainedTokenizerFast
import urllib.request

# KoGPT2 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token=UNK,
            pad_token=PAD, mask_token=MASK)

# 데이터 로드
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv",
    filename="ChatBotData.csv",
)
df = pd.read_csv("ChatBotData.csv")
dataset = Dataset.from_pandas(df)

def get_input(examples):
    inputs = [tokenizer.bos_token + example for example in examples['Q']]
    outputs = [example + tokenizer.eos_token for example in examples['A']]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(outputs, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 데이터셋 전처리
tokenized_datasets = dataset.map(get_input, batched=True)


/root/.cache/pypoetry/virtualenvs/model-training-9TtSrW0h-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Map: 100%|██████████| 11823/11823 [00:00<00:00, 12547.52 examples/s]


In [4]:
# 데이터 크기 줄이기 위함
dataset = tokenized_datasets.train_test_split(test_size=0.01)

# 검증 데이터 분할
train_test_dataset = dataset['test'].train_test_split(test_size=0.2)

train_dataset = train_test_dataset['train']
test_dataset = train_test_dataset['test']

In [5]:
import wandb

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dogcat1943 (nkim123). Use `wandb login --relogin` to force relogin


True

In [6]:
# method
sweep_config = {
    'method': 'random'
}

# hyperparameters
parameters_dict = {
    'epochs': {
        'values': [1]
        },
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-5,
        'max': 7e-5
    },
    'weight_decay': {
        'values': [0.1, 0.3, 0.5]
    },
}

sweep_config['parameters'] = parameters_dict

In [7]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments, EarlyStoppingCallback

def train():
  wandb.init()
  config = wandb.config

  # KoGPT2 모델 로드
  model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")

  # 학습 설정
  training_args = TrainingArguments(
      output_dir='./results',
      num_train_epochs=config.epochs,
      per_device_train_batch_size=1,
      per_device_eval_batch_size=1,
      warmup_steps=500,
      weight_decay=config.weight_decay,
      logging_dir='./logs',
      logging_steps=10,
  )

  # Trainer 설정
  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=test_dataset,
  )

  trainer.train()

  model.save_pretrained('./kogpt2-chatbot')
  tokenizer.save_pretrained('./kogpt2-chatbot')

In [8]:
# 캐시 지우기
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [9]:
sweep_id = wandb.sweep(sweep_config, entity='nkim123', project='minidlthon_kogpt2')

wandb.agent(sweep_id, train, count=1)

wandb.finish()

Create sweep with ID: woqiptq4
Sweep URL: https://wandb.ai/nkim123/minidlthon_kogpt2/sweeps/woqiptq4


wandb: Agent Starting Run: xv0ozbzl with config:
wandb: 	epochs: 1
wandb: 	learning_rate: 2.806447428385206e-05
wandb: 	weight_decay: 0.5
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).


Step,Training Loss
10,15.933200
20,14.015000
30,9.760200
40,2.777700
50,1.328700
60,0.822700
70,0.944100
80,0.549800
90,0.599000


train/epoch,▁▂▃▃▄▅▆▇██
train/global_step,▁▂▃▃▄▅▆▇██
train/grad_norm,▇▇█▂▂▁▁▁▁
train/learning_rate,▁▂▃▄▄▅▆▇█
train/loss,█▇▅▂▁▁▁▁▁
total_flos,6205685760000.0
train/epoch,1.0
train/global_step,95
train/grad_norm,7.28902
train/learning_rate,1e-05
train/loss,0.599


In [10]:
# 저장된 모델 및 토크나이저 로드
trained_model = GPT2LMHeadModel.from_pretrained('./kogpt2-chatbot')
trained_tokenizer = PreTrainedTokenizerFast.from_pretrained('./kogpt2-chatbot')

In [11]:
# 입력 문장 토큰화
input_text = "대답이 너무 빨라"
input_ids = trained_tokenizer.encode(trained_tokenizer.bos_token + input_text + trained_tokenizer.eos_token, return_tensors='pt')

# 모델 추론
outputs = trained_model.generate(input_ids, max_length=50, repetition_penalty=2.0, num_beams=5, early_stopping=True)
output_text = trained_tokenizer.decode(outputs[0], skip_special_tokens=True)

print(output_text)

대답이 너무 빨라


In [12]:
# 파인튜닝 전 모델과 비교
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
tokenizer = PreTrainedTokenizerFast.from_pretrained('skt/kogpt2-base-v2')

# 모델 추론
gen_ids = model.generate(input_ids,
                           max_length=50,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


</s> 대답이 너무 빨라</s> <unk>, "그럼요, 그건 그렇고요." 하고 말했습니다.
그러자 그는 다시 말을 했어요.
'아니, 그럼.'
그는 이렇게 말하고는 고개를 끄덕였지요.
그리고 나서 그가 말한 대로
